In [1]:


import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.layers import Dense, Input, BatchNormalization, Dropout, Concatenate
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.callbacks import ModelCheckpoint
from sklearn import datasets
import numpy as np
import os


2022-12-07 02:04:53.467916: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-07 02:04:53.541832: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-12-07 02:04:53.561802: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-12-07 02:04:53.938999: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; 

In [2]:



%%time
module_url = 'https://tfhub.dev/google/universal-sentence-encoder-large/5'
embed = hub.KerasLayer(module_url, input_shape=[], dtype=tf.string, trainable=True)


2022-12-07 02:04:59.600231: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-07 02:04:59.622123: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-07 02:04:59.622254: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-07 02:04:59.622583: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the approp

CPU times: user 5.63 s, sys: 518 ms, total: 6.15 s
Wall time: 6.23 s


In [ ]:
n = df.course_dept.nunique()
print(n)

In [3]:


def build_model(embed):
    
    model = Sequential([
        Input(shape=[], dtype=tf.string),
        embed,
        Dense(256, activation='relu', kernel_regularizer=tf.keras.regularizers.L2(0.01)),
        Dropout(0.2),
        Dense(190, activation='softmax')
    ])
    model.compile(Adam(1e-4), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    
    return model



In [4]:
model = build_model(embed)
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    (None, 512)               147354880 
                                                                 
 dense (Dense)               (None, 256)               131328    
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 dense_1 (Dense)             (None, 190)               48830     
                                                                 
Total params: 147,535,038
Trainable params: 147,535,038
Non-trainable params: 0
_________________________________________________________________


In [ ]:
import pandas as pd
import sklearn
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
df = pd.read_csv("https://waf.cs.illinois.edu/discovery/course-catalog.csv", )
df.head()
df2 = pd.DataFrame()
df2["text"] = df["Description"]
df2["label"] = preprocessing.LabelEncoder().fit_transform(df["Subject"])

x_train, x_test, y_train, y_test = train_test_split(df2["text"].values, df2["label"].values, test_size=0.2, random_state=42)
x_train = np.asarray(x_train).astype(np.str)

df["Subject"].nunique()

In [ ]:
print(len(x_train))
print(len(y_train))

In [ ]:


model.load_weights('model.h5')

checkpoint = ModelCheckpoint('model.h5', monitor='val_accuracy', save_best_only=True)

history = model.fit(x_train, 
          y_train, 
          batch_size=64, 
          validation_data=(x_test, y_test),
          callbacks=[checkpoint],
          epochs=15)

In [5]:
model.load_weights('model.h5')

aux_model = tf.keras.Model(inputs=model.inputs,
                          outputs=model.outputs + [model.get_layer("keras_layer").output])

final_output, intermediate_layer_output = aux_model.predict(['I want to study computers'])

#print(final_output)
print(len(intermediate_layer_output[0]))

2022-12-07 02:05:10.535706: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory


1/1 [==============================] - 4s 4s/step
512


2022-12-07 02:05:11.335688: I tensorflow/stream_executor/cuda/cuda_blas.cc:1614] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


In [ ]:
model.load_weights('model.h5')
test_pred = model.predict(['I want to study science'])
#listMajors = sorted(os.listdir("/home/dylan/Desktop/batman/CS478-LinguisAquilis/notebooks/datasetMajors/"))
idx = np.argsort(test_pred, axis=1)[:,-5:]
idx = np.asarray(idx, dtype=int, order=None)
for i in idx[0]:
    print(listMajors[i] + " Accuracy: " + str(test_pred[0][i]))

In [6]:
with open('dept_ids.txt', 'r') as dept_ids:
    data = dept_ids.read()
    dept_names = data.split(' ')
    dept_names.remove("\n")
    print(dept_names)

import re
import json


courses = []

registrar_file = open('output.txt')
registrar_file_iter = iter(registrar_file)

for line in registrar_file_iter:
    if line.startswith(tuple(dept_names)):
        course_dict = {'course_dept': '', 'course_number': '', 'course_title':'', 'course_description':'', 'USE_embedding': None}
        course_description = ''
        line_split = line.split(" ")
        course_dict['course_dept'] = line_split[0]
        course_dict['course_number'] = line_split[1]
        course_dict['course_title'] = " ".join(line_split[3:-1])
        line = next(registrar_file_iter)
        while not line.startswith(tuple(dept_names)):
            line.rstrip()
            course_description = course_description+line
            line = next(registrar_file_iter)
        course_description = re.sub('\n' ,'', course_description)          
        course_dict['course_description'] = course_description
        courses.append(course_dict)


['ACCT', 'ADV', 'AFRC', 'AGBU', 'AHA', 'ANTH', 'ARAB', 'ART', 'ARTD', 'ARTE', 'ARTH', 'ARTP', 'ASTR', 'BAN', 'BCOM', 'BIOL', 'BLAW', 'BUAD', 'CE', 'CHEM', 'CHIN', 'CIS', 'CMST', 'COMM', 'CRIM', 'CS', 'DMS', 'DSCI', 'DTAS', 'DTHY', 'DVT', 'ECE', 'ECHO', 'ECON', 'EDUC', 'ENG', 'ENGR', 'EXSC', 'FIN', 'FREN', 'GENS', 'GEOG', 'GEOL', 'GERM', 'GERO', 'GLST', 'GNDR', 'HA', 'HI', 'HIST', 'HONS', 'HP', 'HUM', 'IEP', 'IME', 'IPH', 'JPN', 'JRN', 'KIN', 'LATN', 'LIBA', 'MATH', 'ME', 'MFET', 'MKTG', 'MNGT', 'MS', 'MUS', 'NURS', 'NUTR', 'OTA', 'PET', 'PH', 'PHIL', 'PHYS', 'POLS', 'PRFS', 'PRL', 'PSY', 'RADT', 'RELS', 'REST', 'RTV', 'SOC', 'SOCW', 'SPAN', 'SPTM', 'STAT', 'STEM', 'TECH', 'THTR', 'UNIV', 'WLC']


In [7]:
descriptions = []

aux_model = tf.keras.Model(inputs=model.inputs,
                          outputs=model.outputs + [model.get_layer("keras_layer").output])


for course in courses:
    
    course_description = (course['course_description'])
    descriptions.append(course_description)
    classification, course_embedding = aux_model.predict([course_description])
    course['USE_embedding'] = course_embedding[0]

1/1 [==============================] - 0s 34ms/step


1/1 [==============================] - 0s 33ms/step


1/1 [==============================] - 0s 35ms/step


1/1 [==============================] - 0s 32ms/step


1/1 [==============================] - 0s 33ms/step


1/1 [==============================] - 0s 31ms/step


In [ ]:
print(courses[0])

In [8]:
course_embeddings = []

for course in courses:
    course_embeddings.append(course['USE_embedding'])
    
print(course_embeddings[0])

[-5.04226685e-02  1.84164159e-02 -1.07048368e-02 -4.99983504e-02
 -6.68426156e-02  3.14607807e-02  9.49538052e-02 -2.67407447e-02
 -7.02940999e-03  6.24741940e-03 -6.19863719e-02  6.76637292e-02
 -5.98934926e-02 -5.83909824e-02 -1.09606590e-02 -7.34780356e-03
 -5.57523854e-02 -2.95216572e-02 -7.87957981e-02 -5.37563227e-02
  6.40814519e-03  2.93750893e-02  2.75393017e-03  4.20090780e-02
  4.47656214e-02 -2.00677160e-02  2.07196455e-02 -8.04579407e-02
 -3.33292373e-02  1.81298959e-03 -5.91635890e-02 -1.96720622e-02
  2.33530644e-02  3.16280574e-02  9.08539519e-02 -4.73106839e-02
 -1.90348383e-02  6.93468982e-03 -6.87728450e-02 -8.26714095e-03
 -5.41944653e-02 -3.19730267e-02  3.17852870e-02 -2.61977930e-02
 -1.46754934e-02  9.92827490e-03  2.65208725e-02  9.74543169e-02
  1.38808712e-02 -3.69201228e-02  3.89055945e-02 -2.98332609e-02
  6.85217651e-03 -4.12079506e-02  7.58962631e-02 -3.63258296e-04
  1.07904375e-02  7.71278217e-02  1.08049195e-02  1.03903254e-02
  1.91341504e-03  8.05048

In [9]:
import hnswlib
import numpy as np
import pickle

dim = 512
num_elements = len(courses)
print("length of courses is: " + str(num_elements))

# Generating sample data
data = course_embeddings
ids = np.arange(num_elements)

# Declaring index
p = hnswlib.Index(space = 'cosine', dim = dim) # possible options are l2, cosine or ip

# Initializing index - the maximum number of elements should be known beforehand
p.init_index(max_elements = num_elements, ef_construction = 200, M = 16)

# Element insertion (can be called several times):
p.add_items(data, ids)

# Controlling the recall by setting ef:
p.set_ef(50) # ef should always be > k

for idx, course in enumerate(courses):
    print('COURSE ' + course['course_dept'] + ' ' + course['course_number'] + ' - ' + course['course_title'])
    classification, course_embedding = aux_model.predict([course['course_description']])
    labels, distances = p.knn_query(course_embedding[0], k = 5)
    for i in labels[0]:
        print(courses[i]['course_title'])
    print("\n\n")
        
p.save_index('new_index.bin')



length of courses is: 844
COURSE ACCT 201 - Accounting Principles I
1/1 [==============================] - 0s 30ms/step
Accounting Principles I
Accounting Ethics & Compliance
Special Problems in Industrial Supervision or Advanced
Sport Finance and Economics
Strategies for Writers



COURSE ACCT 303 - Intermediate Accounting I
1/1 [==============================] - 0s 31ms/step
Intermediate Accounting I
Introduction to Federal Income Taxation
Auditing Theory and Practice
Financial Report and Analysis
Alternative Investments: Uses and Recording



COURSE ACCT 311 - Introduction to Federal Income Taxation
1/1 [==============================] - 0s 33ms/step
Introduction to Federal Income Taxation
Auditing Theory and Practice
Intermediate Accounting I
Financial Report and Analysis
Taxation of Corporations and Partnerships



COURSE ACCT 390 - Individual Taxation Practicum
1/1 [==============================] - 0s 33ms/step
Individual Taxation Practicum
Practice Management, Ethics, and Juris

Human Osteology
Physical Anthropology Seminar
Archaeology Lab Methods
Cultural Anthropology Seminar
Introduction to Forensic Anthropology



COURSE ANTH 370 - Cultural Anthropology Seminar
1/1 [==============================] - 0s 34ms/step
Cultural Anthropology Seminar
Physical Anthropology Seminar
History of Anthropological Thought
Archaeology Lab Methods
Human Osteology



COURSE ANTH 390 - Physical Anthropology Seminar
1/1 [==============================] - 0s 33ms/step
Physical Anthropology Seminar
Cultural Anthropology Seminar
History of Anthropological Thought
Human Osteology
Archaeology Lab Methods



COURSE ANTH 401 - Archaeology Lab Methods
1/1 [==============================] - 0s 32ms/step
Archaeology Lab Methods
Introduction to Forensic Anthropology
Cultural Anthropology Seminar
History of Anthropological Thought
Human Osteology



COURSE ANTH 475 - History of Anthropological Thought
1/1 [==============================] - 0s 32ms/step
History of Anthropological Thought
Cul

1/1 [==============================] - 0s 34ms/step
Medieval Art
Select Topics in Art History
Special Studies in Art History
Art History II
Seminar in Art Education



COURSE ARTH 343 - Renaissance Art
1/1 [==============================] - 0s 34ms/step
Renaissance Art
Introduction to German Literature
Special Topics in German
Beginning German II
History of Philosophy I



COURSE ARTH 353 - 19th Century Art
1/1 [==============================] - 0s 33ms/step
19th Century Art
Select Topics in Art History
Art History II
Painting I
History of World Drama I



COURSE ARTH 359 - Special Studies in Art History
1/1 [==============================] - 0s 34ms/step
Special Studies in Art History
Art History II
Medieval Art
Seminar in Art Education
Select Topics in Art History



COURSE ARTH 390 - Select Topics in Art History
1/1 [==============================] - 0s 31ms/step
Select Topics in Art History
Art History II
Medieval Art
Special Studies in Art History
Seminar in Art Education



COURS

1/1 [==============================] - 0s 32ms/step
Independent Research
Advanced Pulmonary Diseases
Practicum Seminar B

Clinical Practice of Respiratory Therapy I



COURSE BAN 305 - Data Analytics for Business
1/1 [==============================] - 0s 33ms/step
Data Analytics for Business
Organizational Theory and Design
Business Processes and Information Systems
Small Business Consulting
Seminar in Contemporary Marketing Problems



COURSE BCOM 331 - Corporate & Managerial Communication
1/1 [==============================] - 0s 35ms/step
Corporate & Managerial Communication
Internship in Journalism
Co-op Experience
Field Experience
Computer Skills for Non-Business Majors



COURSE BCOM 401 - Business Career Planning and Professional Development
1/1 [==============================] - 0s 31ms/step
Business Career Planning and Professional Development
Entrepreneurship Feasibility Analysis and Business Plan
Small Business Consulting
Services Marketing
Corporate & Managerial Communicati

1/1 [==============================] - 0s 34ms/step
Mass Communication Research
Social Media Journalism
Comparative Structures of Mass Media in a Global Community
Independent Project
Social Media Communication



COURSE CMST 101 - Introduction to Public Speaking
1/1 [==============================] - 0s 34ms/step
Introduction to Public Speaking
Cultural Awareness
Field Experience
Honors Seminar
Seminar in Public Relations



COURSE CMST 201 - Introduction to Communication Studies
1/1 [==============================] - 0s 35ms/step
Introduction to Communication Studies
Rhetoric and Diversity
History of Rhetoric
Communication and Workplace Relationships
Communication and Healthcare



COURSE CMST 204 - Business and Professional Communication
1/1 [==============================] - 0s 32ms/step
Business and Professional Communication
Seminar in Interpersonal Communication
Intercultural Communication
Seminar in Organizational Communication
Organizational Communication



COURSE CMST 286 - V

...

1/1 [==============================] - 0s 34ms/step
Orientation to Radiologic and Imaging Sciences
Radiographic Procedures II
Special Topics in Radiologic and Imaging Sciences
Digital Radiography and Imaging Informatics
Professional Development and Literacy in Imaging Sciences



COURSE RADT 307 - Critical Thinking Skills in Radiography
1/1 [==============================] - 0s 33ms/step
Critical Thinking Skills in Radiography
Professional Development and Literacy in Imaging Sciences
Radiologic and Imaging Sciences Clinical Education Methods
Introduction to Radiologic Technology
Magnetic Resonance Imaging and Computed Tomography Procedures



COURSE RADT 311 - Introduction to Radiologic Technology
1/1 [==============================] - 0s 34ms/step
Introduction to Radiologic Technology
Digital Radiography and Imaging Informatics
Radiologic Physics
Magnetic Resonance Imaging and Computed Tomography Procedures
Special Topics in Radiologic and Imaging Sciences



COURSE RADT 313 - Radiogr

1/1 [==============================] - 0s 33ms/step
Management and Leadership in Respiratory Therapy
Abdominal Procedures
Pediatric and Newborn Respiratory Therapy
Cardiopulmonary Pharmacology
Organizational Behavior and Strategic Planning in Healthcare



COURSE REST 453 - Respiratory Therapy Disease Management
1/1 [==============================] - 0s 33ms/step
Respiratory Therapy Disease Management
Independent Study of Respiratory Therapy
Pulmonary Rehabilitation and Geriatrics
Introduction to DMS Principles and Procedures
Cardiopulmonary Pharmacology



COURSE REST 455 - Pulmonary Rehabilitation and Geriatrics
1/1 [==============================] - 0s 33ms/step
Pulmonary Rehabilitation and Geriatrics
Respiratory Therapy Disease Management
Independent Study of Respiratory Therapy
Clinical Practice I-Vascular
Health Assessment for Registered Nurses



COURSE REST 460 - Independent Study of Respiratory Therapy
1/1 [==============================] - 0s 34ms/step
Independent Study of Re

1/1 [==============================] - 0s 33ms/step
Internship in Sociology
Special Topics in Spanish
The Hispanic Essay
Spanish Linguistics II
Cultural Contexts for Conversation



COURSE SPAN 203 - Intermediate Spanish I
1/1 [==============================] - 0s 33ms/step
Intermediate Spanish I
Beginning Chinese II
Advanced Spanish I
Intermediate French I
Intermediate Chinese II



COURSE SPAN 205 - Cultural Contexts for Conversation
1/1 [==============================] - 0s 32ms/step
Cultural Contexts for Conversation
Spanish Linguistics II
Special Topics in Spanish
Composition and Stylistics
The Hispanic Essay



COURSE SPAN 290 - Special Topics in Spanish
1/1 [==============================] - 0s 31ms/step
Special Topics in Spanish
Cultural Awareness Training
History of English Languages
German for the Professional
Spanish for the Professional



COURSE SPAN 306 - Advanced Spanish I
1/1 [==============================] - 0s 34ms/step
Advanced Spanish I
Intermediate Chinese II
Begi

1/1 [==============================] - 0s 34ms/step
Materials & Manufacturing Processes
Semiconductors
Strength of Materials
Geometric Dimensioning and Tolerancing
Machine Design



COURSE TECH 261 - Fundamentals of Instrumentation
1/1 [==============================] - 0s 35ms/step
Fundamentals of Instrumentation
Vibrations
Geometric Dimensioning and Tolerancing
Machine Design
Mechanical Engineering Design



COURSE TECH 275 - Statistics and Safety in Manufacturing
1/1 [==============================] - 0s 35ms/step
Statistics and Safety in Manufacturing
Mathematical Statistics
Work Cell and Production Systems Design
Global Economic Issues
Public and Nonprofit Personnel Administration



COURSE TECH 299 - Co-op Experience
1/1 [==============================] - 0s 32ms/step
Co-op Experience
Engineering Orientation
Solid Modeling
Applied Problem Solving
Manufacturing



COURSE TECH 315 - Strength of Materials
1/1 [==============================] - 0s 32ms/step
Strength of Materials
Mate

Cultural Awareness
Introduction to Literature
Introduction to Public Speaking
Special Topics in French
Special Topics in German



COURSE WLC 337 - The African Diaspora in Latin American Literature and Culture
1/1 [==============================] - 0s 32ms/step
The African Diaspora in Latin American Literature and Culture
Government and Politics of Latin America
The Humanistic Tradition in Literature II
African American History to 1865
Introduction to African Literatures



COURSE WLC 422 - Cultural Awareness Training
1/1 [==============================] - 0s 33ms/step
Cultural Awareness Training
Spanish for the Professional
German for the Professional
Special Topics in Spanish
History of English Languages



COURSE WLC 460 - Internship in Language
1/1 [==============================] - 0s 32ms/step
Internship in French
Internship in Language
Special Topics in French
Special Topics in German
Intermediate Chinese II





In [10]:
for idx, course in enumerate(courses):
    print('COURSE ' + course['course_dept'] + ' ' + course['course_number'] + ' - ' + course['course_title'])
    classification, course_embedding = aux_model.predict([course['course_title']])
    labels, distances = p.knn_query(course_embedding[0], k = 5)
    for i in labels[0]:
        print(courses[i]['course_title'])
    print("\n\n")

COURSE ACCT 201 - Accounting Principles I
1/1 [==============================] - 0s 38ms/step
Intermediate Accounting I
Introduction to Federal Income Taxation
Auditing Theory and Practice
Alternative Investments: Uses and Recording
Financial Report and Analysis



COURSE ACCT 303 - Intermediate Accounting I
1/1 [==============================] - 0s 30ms/step
Intermediate Accounting I
Introduction to Federal Income Taxation
Auditing Theory and Practice
Alternative Investments: Uses and Recording
Financial Report and Analysis



COURSE ACCT 311 - Introduction to Federal Income Taxation
1/1 [==============================] - 0s 31ms/step
Principles of Microeconomics
Labor Economics
Development Economics
Applied Cost-Benefit Analysis
Business Finance



COURSE ACCT 390 - Individual Taxation Practicum
1/1 [==============================] - 0s 30ms/step
Management Information Systems
Individual Taxation Practicum
Operations Management
Sport Finance and Economics
Introduction to Federal Inco

1/1 [==============================] - 0s 32ms/step
Archaeology of Eastern North America
Peoples of Africa
Cultural Anthropology Seminar
Physical Anthropology Seminar
History of Anthropological Thought



COURSE ANTH 401 - Archaeology Lab Methods
1/1 [==============================] - 0s 32ms/step
Human Osteology
Archaeology of Eastern North America
Introduction to Forensic Anthropology
Primatology
Cultural Anthropology Seminar



COURSE ANTH 475 - History of Anthropological Thought
1/1 [==============================] - 0s 32ms/step
Cultural Anthropology Seminar
History of Anthropological Thought
Physical Anthropology Seminar
Archaeology Lab Methods
Archaeology of Eastern North America



COURSE ANTH 499 - Internship in Anthropology
1/1 [==============================] - 0s 31ms/step
Cultural Anthropology Seminar
History of Anthropological Thought
Archaeology of Eastern North America
Physical Anthropology Seminar
Archaeology Lab Methods



COURSE ARAB 102 - Beginning Arabic II
1/1 [==

1/1 [==============================] - 0s 40ms/step
Art History II
Select Topics in Art History
Seminar in Art Education
Special Studies in Art History
Medieval Art



COURSE ARTH 359 - Special Studies in Art History
1/1 [==============================] - 0s 41ms/step
Select Topics in Art History
Medieval Art
Art History II
Special Studies in Art History
Seminar in Art Education



COURSE ARTH 390 - Select Topics in Art History
1/1 [==============================] - 0s 39ms/step
Select Topics in Art History
Medieval Art
Art History II
Special Studies in Art History
19th Century Art



COURSE ARTP 211 - Introduction to Photography
1/1 [==============================] - 0s 43ms/step
Graphic Design IV
Printmaking IV
Materials and Methods in Elementary and Middle School Art Education
Sculpture II
Illustration Techniques



COURSE ARTP 311 - Advanced Darkroom Photography
1/1 [==============================] - 0s 42ms/step
Introduction to Photography
Painting I
Printmaking IV
Printmaking II


1/1 [==============================] - 0s 43ms/step
Statistics and Safety in Manufacturing
Managing Warehousing and Transportation
Project Management 2
Small Business Consulting
Public and Nonprofit Budgeting and Finance



COURSE BLAW 363 - Law for the Entrepreneur
1/1 [==============================] - 0s 43ms/step
Human Resource Management
Public and Nonprofit Personnel Administration
Seminar in Human Resource Management
Strategic Writing
Project Management 2



COURSE CHEM 107 - Elements in Everyday Chemistry
1/1 [==============================] - 0s 43ms/step
Inorganic Chemistry
Organic Chemistry II
Chemistry Seminar II
Elements in Everyday Chemistry
Principles of Chemistry



COURSE CHEM 141 - Principles of Chemistry
1/1 [==============================] - 0s 42ms/step
Organic Chemistry II
Elements in Everyday Chemistry
Inorganic Chemistry
Chemistry Seminar II
General Chemistry I



COURSE CHEM 143 - Survey of Organic and Biochemistry
1/1 [==============================] - 0s 42ms

1/1 [==============================] - 0s 39ms/step
Communication and Workplace Relationships
Rhetoric and Diversity
Introduction to Communication Studies
Organizational Communication
Intercultural Communication



COURSE CMST 286 - Visual Rhetoric and Culture
1/1 [==============================] - 0s 45ms/step
Non-Western World Literature
Shakespeare's Comedies and Romances
Advanced Darkroom Photography
Literature of the Middle Ages
Concepts of Good and Evil in Literature



COURSE CMST 302 - Gender, Communication, and Culture
1/1 [==============================] - 0s 43ms/step
Non-Western World Literature
Topics in Gender Studies
Concepts of Good and Evil in Literature
an overview of the main currents in the development of American literature, from its
Gender Studies Independent Study



COURSE CMST 308 - Rhetoric and Diversity
1/1 [==============================] - 0s 45ms/step
History of Rhetoric
Rhetoric and Diversity
Introduction to Women in Literature
Gender, Communication, and 

...

1/1 [==============================] - 0s 37ms/step
Social Psychology
Research Methods and Statistics I
Orientation to a Major in Psychology
Internship
Cognitive Psychology



COURSE PSY 303 - Research Methods and Statistics I
1/1 [==============================] - 0s 33ms/step
Introduction to Categorical Data Analysis
Applied Regression Analysis
Sampling Theory & Applications
Principles of Microeconomics
Mathematical Statistics



COURSE PSY 312 - Sensation and Perception
1/1 [==============================] - 0s 34ms/step
Sensation and Perception
Cognitive Psychology
392 or TECH 331); STAT 255 
Special Topics in Psychology




COURSE PSY 325 - Tests and Measures
1/1 [==============================] - 0s 34ms/step

The Soviet Union
International Law
U.S. Military History I (1740-1900)
Sensation and Perception



COURSE PSY 353 - Research Methods and Statistics II
1/1 [==============================] - 0s 35ms/step
Introduction to Categorical Data Analysis
Applied Regression Analysis
S

1/1 [==============================] - 0s 31ms/step
Seminar in Medicine
Population Health and Professional Nursing Practice
Advanced Patient Care for Imaging Sciences
Radiographic Procedures IV
Health Assessment



COURSE RADT 334 - Radiologic Physics
1/1 [==============================] - 0s 33ms/step
Mineralogy
General Astronomy
CT/MRI Physics and Instrumentation
Structural Geology
Undergraduate Teaching in Geology



COURSE RADT 316, - 334, RADT 343, RADT 411, and RADT 412 
1/1 [==============================] - 0s 34ms/step
Introduction to Women in Literature
History of Rhetoric
Beginning Arabic II
Gender, Communication, and Culture
Hispanic Cultures



COURSE RADT 316, - 334, RADT 343, RADT 411, and RADT 412 
1/1 [==============================] - 0s 32ms/step
Introduction to Women in Literature
History of Rhetoric
Beginning Arabic II
Gender, Communication, and Culture
Hispanic Cultures



COURSE RADT 345 - Radiographic Pathology
1/1 [==============================] - 0s 32ms/step

1/1 [==============================] - 0s 32ms/step
Introduction to Social Justice
Criminology
Sociology of the Environment
Population
Sociology Research Design



COURSE SOCW 239 - Human Behavior and the Social Environment
1/1 [==============================] - 0s 32ms/step
Marriage and Family
Research Methods and Statistics II
Death, Dying, and Bereavement
Understanding Evolution
Sociological Aspects of Aging



COURSE SOCW 242 - Introduction to Substance Abuse and Addictive Disorders
1/1 [==============================] - 0s 37ms/step
Care of Children and Families
Professional and Current Issues in Oral Healthcare
Internship in Health Administration I
Internship in Professional Nursing
Personal and Professional Development



COURSE SOCW 325 - Introduction to Social Work Statistics
1/1 [==============================] - 0s 34ms/step
Introduction to Social Work Statistics
Social Work Practice with Micro Systems I
Social Work Practicum II
Social Work Practice with Macro Systems
Capsto

1/1 [==============================] - 0s 32ms/step
Spanish Linguistics II
Composition and Stylistics
Special Topics in Spanish
Cultural Contexts for Conversation
The Hispanic Essay



COURSE SPAN 415 - The Hispanic Essay
1/1 [==============================] - 0s 31ms/step
Introduction to Arabic Literature
Literature of the Middle Ages
Beginning Arabic II
US Latino Literature and Culture
Chaucer



COURSE SPAN 454 - Golden Age Drama
1/1 [==============================] - 0s 31ms/step
Ancient and Early Imperial China
an overview of the main currents in the development of American literature, from its
England to 1600
Comparative Criminal Justice Systems
American Cinema and Culture



COURSE SPAN 490 - Special Topics in Spanish
1/1 [==============================] - 0s 32ms/step
Spanish Linguistics II
Special Topics in Spanish
Cultural Contexts for Conversation
Composition and Stylistics
The Hispanic Essay



COURSE SPTM 223 - Principles of Recreation
1/1 [==============================] 

1/1 [==============================] - 0s 33ms/step
Professional Studies Internship
Painting I
Advanced Darkroom Photography
Introduction to Photography
Painting IV



COURSE THTR 102 - Introduction to Theatrical Production
1/1 [==============================] - 0s 32ms/step
History of Theatre I
Stage Management
Independent Study in Theatre
Fundamentals of Scene Study
Lighting Technology



COURSE THTR 150 - Practicum in Dance
1/1 [==============================] - 0s 31ms/step
Practicum in Dance
Intermediate Dance Practicum
Practicum in Dance, Advanced Level
Advanced Lighting Design
USI Jazz Ensemble



COURSE THTR 200 - Independent Study in Theatre
1/1 [==============================] - 0s 31ms/step
Stage Management
Independent Study in Theatre
History of Theatre I
Theatre Laboratory II
Lighting Technology



COURSE THTR 224 - Production Technology
1/1 [==============================] - 0s 31ms/step
Scene Design
Computer Basics
Advanced Scene Design
Special Design Projects
Manufactur

In [ ]:
import hnswlib
import numpy as np
import pickle

dim = 512
num_elements = len(courses)
print("length of courses is: " + str(num_elements))

# Generating sample data
data = course_embeddings
ids = np.arange(num_elements)

# Declaring index
p = hnswlib.Index(space = 'cosine', dim = dim) # possible options are l2, cosine or ip

# Initializing index - the maximum number of elements should be known beforehand
p.init_index(max_elements = num_elements, ef_construction = 200, M = 16)

# Element insertion (can be called several times):
p.add_items(data, ids)

# Controlling the recall by setting ef:
p.set_ef(50) # ef should always be > k


classification, course_embedding = aux_model.predict(['I want to study engineering'])
labels, distances = p.knn_query(course_embedding[0], k = 5)
for i in labels[0]:
    print(courses[i]['course_title'])
        




In [ ]:
import hnswlib
import numpy as np
import pickle


p = hnswlib.Index(space = 'cosine', dim = 512) # possible options are l2, cosine or ip
p.load_index("/home/dylan/Desktop/batman/CS478-LinguisAquilis/notebooks/indexUSETrained.bin")
for idx, course in enumerate(courses):
    print('COURSE ' + course['course_dept'] + ' ' + course['course_number'] + ' - ' + course['course_title'])
    classification, course_embedding = aux_model.predict([course['course_title']])
    labels, distances = p.knn_query(course_embedding[0], k = 5)
    print(labels[0])
    for i in labels[0]:
        print(courses[i]['course_title'])
    print("\n\n")
        

In [ ]:
p.lo